In [52]:
%run -i ../python/common.py

closeAllOpenTtySessions()
bash = BashSession()

Exception ignored in: <function BashSession.__del__ at 0x7ff0e7b752d0>
Traceback (most recent call last):
  File "/opt/app-root/src/openos/content/python/common.py", line 868, in __del__
    bashSessionClose(self.session)
  File "/opt/app-root/src/openos/content/python/common.py", line 847, in bashSessionClose
    closeTtySession(session)
  File "/opt/app-root/src/openos/content/python/common.py", line 618, in closeTtySession
    TtyOpenSessions.remove(session)
ValueError: list.remove(x): x not in list


(cont:gs:purpose)=
# Purpose of operating systems

The purpose of an operating system is to provide everything needed to enable applications to run on computers.  We [first](cont:gs:purpose:hw) present a simple model of hardware and why the OS is needed, then discuss how this has resulted in operating systems becoming the [fundamental platform](cont:gs:purpose:platform) that all computing depends on, and finally discuss why we, as operating system researchers and developers, feel like the [fun has just started](cont:gs:purpose:care). Our hope is that at the end of this chapter you will not only realize why it is so important for all computer scientists and engineers to have a fundamental understanding of operating systems, but to also to realize why so many people are excited to work on and contribute to operating systems.  Consider that today's dominant operating systems are open source, and rely on 1000s of contributors from all over the world, many who volunteer their time.



(cont:gs:purpose:hw)=
## A simple model of hardware

The fundamental job of the operating system is to enable applications to run on a computer.  To understand why this is complicated, consider the very simple model of a computer depicted in {numref}`simplecomputer-fig`.  We briefly describe each of the key components of this computer, and what you need to know for now.

```{figure} ../images/intro/hardware.png
---
width: 50% 
name: simplecomputer-fig
---
A simple model of a computer.  The CPU is connected to high speed memory, and through a lower speed bus to a network controller and disk controller that are in turn connected to a network (ethernet in this case) and a disk. 
```

- **Central Processing Unit (CPU)**:  The smart part that executes instructions is called the Central Processing Unit (CPU) or **processor**.  In our simple model, these instructions operate by modifying the contents of various processor registers, and loading new register contents from memory, and storing registers into memory. 

- **Memory:** Also referred to as **Random Access Memory (RAM)**, holds the instructions and data that running applications require.  It is physically connected to the CPU via direct wiring.  Random access here refers to the fact that the processor can directly load or store any memory location (addressed as an 8-bit byte) in the same amount of time.   Memory today is normally volatile, where the contents are lost when the computer is powered off. 

- **I/O bus:** If the computer wasn't connected to anything, you couldn't do much with it.  All computers have some form of Input/Output (or I/O) bus that connects them to different devices.  While there are many kinds of I/O devices, for now we will ignore everything but: 1) storage, and 2) networks.   The processor normally talks to an I/O device by, typically, loading and storing registers on a controller specific to the device, and that controller in turn communicates with the corresponding device(s) and transfers data between the device and memory. 

- **Storage:** Since the memory is typically volatile, computers have to have some form of storage that are non-volatile.  This may include hard drives, SSDs, optimical drives, usb sticks, etc.  Most storage devices are accessed at a block granularity (e.g., 512 or 4096 bytes), and where entire blocks are transferred between storage and memory. Most storage devices are not **Random Access**, for example, technologies like a hard disk drive require the disk head to physically move to the location on the disk before reading or writing the data.  When the processor wants to read or write a block of disk, it tells the storage controller the block of disk to transfer to or from memory, and the storage controller notifies the processor when the operation is complete. 

- **Networking:** To talk to the outside world computers are normally connected to some network, typically today ethernet, where information is transferred between memory and the network in some form of packet as shown in {numref}`ethernet-packet`.  When the processor wants to send a message over the network, it is prepared in memory, and then tells the network controller to send it.  When new packets arrive over the network, the controller copies them into memory, and then tells the processor (in some fashion) that a new packet is available to be processed. 

```{figure} ../images/intro/ethernet-frame.png
---
width: 80% 
name: ethernet-packet
---
A ethernet packet is organized into a frame, that includes the address of the sender, the destination of the packet, the length, and the data or payload and a 32-bit hash (the CRC) of the rest of the contents that can be used to check of the packet has been corrupted in transit.  
```

```{Note} Note this model is highly abstracted and very simple.  As we will learn later in the course:
- modern processors can execute hundreds of instructions in parallel, and modern computers can be composed of many processors,
- memory today is organized in Non-Uniform Memory Access (NUMA) where some memory is closer to some processors; so its not really random access
- memory is today very slow compared to the processor, and normally there are multiple layers of **cache** so that the CPU doesn't have to wait for the slow memory every time it needs to fetch an instruction to execute.
- etc...
```










(cont:gs:purpose:platform)=
## The fundamental platform

What makes operating systems so interesting is that they are the fundamental platform on which all other software is written. We briefly discuss each of their core tasks; basically this is a summary of the entire course in one simple section. 

```{figure} ../images/intro/purpose-OS-source-03.drawio.png
---
width: 50% 
name: mappingint-fig
figclass: margin
---
Providing interfaces that applications can use on top of the complicated interfaces of today's hardware. 
```

(cont:gs:purpose:platform:abs)=
### Abstracting Hardware

If applications needed to understand all the complicated hardware described [above](cont:gs:purpose:hw), many fewer applications would  be written.  Consider, for example storage, the abstraction provided by the hardware are blocks.  Applications that persist data want abstractions of directories and files.  

As shown in {numref}`mappingint-fig`, the most important job of the operating system is to enable a set of higher level abstractions, often called "virtual abstractions" on top of the low-level primitive "physical abstractions" provided by the hardware.  That is, the operating system  translates physical abstractions to virtual/clean ones that applications can be written to.  Key abstractions we will discuss include:
- Process: An abstraction of a CPU or processor, where every time a **program** is started it executes as a separate process. 
- File system: An abstraction of storage providing structure for organizing non-volatile data, for example, a hierarchical directory structure containing files.
- Virtual memory: An abstraction of the memory that a process can access.

```{admonition} Process versus Program
It is important to understand the difference between programs and processes.  When a user want to run an application, or **program**, the operating system creates a new process and loads the program instructions from storage into memory for the process executes.  There can be multiple processes executing the same program that running at the same time.  For example, while I am writing this book, when I type "ps -e" to tell me all the processes currently running, (see margin) I find that I have multiple "bash" shells running at the same time, a large number of "python" programs, and of course the "ps" program itself. We will talk more later about what the other parts of this command means.
```

In [53]:
bash.run("ps -e")

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

```{figure} ../images/intro/purpose-OS-source-04.drawio.png
---
width: 3in 
name: multipleapps-fig
align: right
---
Supporting many applications 
```

(cont:gs:purpose:platform:multi)=
### Share Hardware

Modern operating systems enable many (thousands) of applications to run at the same time ({numref}`multipleapps-fig`).  The operating system needs to ensure that applications can't break each other.   For example, the operating system tries to ensure that a broken or malicious application (really process running the application program) can't modify the memory of keep other processes from using their fair share of resources.  This is especially the case because different applications may have very different requirements for example, your editor needs to be responsive to you typing characters, while if you compile a program, the compiler tries to use as much processing and disk as possible.  The operating system should ensure that the compiler doesn't destroy the interactivity of your editor.   

(cont:gs:purpose:platform:hwdiv)=
### Enabling diverse hardware

Operating systems often run on diverse hardware, with computers that range from embedded ARM processors on your watch, to massive High Performance Computing systems ({numref}`hw-fig`). 

```{figure} ../images/intro/purpose-OS-source-06.drawio.png
---
width: 50% 
name: hw-fig
figclass: margin
---
Different kinds of computers 
```

The thing that makes it possible to introduce new hardware is that the operating system enables a common set of abstractions across these different systems.  That is the reason why, for example, Netflix runs on my mac, ipad, and tesla.  The operating system provides a common set of abstractions so that application programmers don't have to re-write everything when moving from one system to another.  The reason it is possible to introduce new processors (e.g., ARM, Intel, Power), and new computers (e.g., servers, laptops, cell phones) is that operating systems hide most of the complexity from the application developer. 

(cont:gs:purpose:platform:security)=
### Supporting many users 


```{figure} ../images/intro/purpose-OS-source-05.drawio.png
---
width: 100% 
name: sec-fig
figclass: margin
---
From different users 
```

(cont:gs:purpose:platform:sec-outside)=
### Protecting external attacks

```{figure} ../images/intro/purpose-OS-source-07.drawio.png
---
width: 100% 
name: secoutside-fig
figclass: margin
---
Protecting users from outside world.
```



(cont:gs:purpose:platform:multi)=
### Mutliplexing Hardware

```{figure} ../images/intro/purpose-OS-source-08.drawio.png
---
width: 100% 
name: secoutside-fig
figclass: margin
---
Protecting users from outside world.
```

```{figure} ../images/intro/purpose-OS-source-12.drawio.png
---
width: 100% 
name: secoutside-fig
figclass: margin
---
Supporting devices
```




(cont:gs:purpose:platform:multi)=
### Mutliplexing Hardware




Talk about how support many kinds of hardware, PPC, intel, different disks, different storage... application doesn't need to change.



- abstracting the HW - providing interfaces that applications can use
   - 
   - support many different types of computers 
- mutiplexing hardware - sharing
  - allow many applications to share resources  
  - supporting requires, e.g., real time... 
- security:                                                                                 
  - protect users from each other                                                           
  - protect against attacks                                                                 
- enable control/configuration 
  - tuning
  - customization 
  - specialization
- enable devices - not common abstracion                                                                                                   
- summarize tasks of the operating system

(cont:gs:purpose:care)=
## Why do we care
- core platform everything else relies on, toughest code to write
- means that everything relies on it

- is the fun over... talk about future/changes
